<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Goal" data-toc-modified-id="Goal-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Goal</a></span></li><li><span><a href="#Var" data-toc-modified-id="Var-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Var</a></span></li><li><span><a href="#Init" data-toc-modified-id="Init-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Init</a></span></li><li><span><a href="#Converting-qiime2-files" data-toc-modified-id="Converting-qiime2-files-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Converting qiime2 files</a></span><ul class="toc-item"><li><span><a href="#OTU-table" data-toc-modified-id="OTU-table-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>OTU table</a></span></li><li><span><a href="#Taxonomy" data-toc-modified-id="Taxonomy-4.2"><span class="toc-item-num">4.2&nbsp;&nbsp;</span>Taxonomy</a></span></li><li><span><a href="#Tree" data-toc-modified-id="Tree-4.3"><span class="toc-item-num">4.3&nbsp;&nbsp;</span>Tree</a></span></li></ul></li><li><span><a href="#Load" data-toc-modified-id="Load-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Load</a></span><ul class="toc-item"><li><span><a href="#Checks" data-toc-modified-id="Checks-5.1"><span class="toc-item-num">5.1&nbsp;&nbsp;</span>Checks</a></span></li></ul></li><li><span><a href="#Formatting-metadata" data-toc-modified-id="Formatting-metadata-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Formatting metadata</a></span></li><li><span><a href="#Checking-overlap" data-toc-modified-id="Checking-overlap-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>Checking overlap</a></span><ul class="toc-item"><li><span><a href="#Filtering-to-intersection" data-toc-modified-id="Filtering-to-intersection-7.1"><span class="toc-item-num">7.1&nbsp;&nbsp;</span>Filtering to intersection</a></span></li></ul></li><li><span><a href="#Making-phyloseq-obj" data-toc-modified-id="Making-phyloseq-obj-8"><span class="toc-item-num">8&nbsp;&nbsp;</span>Making phyloseq obj</a></span><ul class="toc-item"><li><span><a href="#Writing-object" data-toc-modified-id="Writing-object-8.1"><span class="toc-item-num">8.1&nbsp;&nbsp;</span>Writing object</a></span></li></ul></li><li><span><a href="#sessionInfo" data-toc-modified-id="sessionInfo-9"><span class="toc-item-num">9&nbsp;&nbsp;</span>sessionInfo</a></span></li></ul></div>

# Goal

* Creating phyloseq R object from LLA (qiime2) files
* The phyloseq file will be used for initial data exploration and dataset tidying

# Var

In [1]:
work_dir = '/ebio/abt3_projects/Georg_animal_feces/data/16S/LLA/phyloseq/'

# qiime2 files
LLA_dir = '/ebio/abt3_projects/Georg_animal_feces/data/16S/LLA/'
SV_file = file.path(LLA_dir, 'table_merged_filt.qza')
tax_file = file.path(LLA_dir, 'taxonomy.qza')
tree_file = file.path(LLA_dir, 'aligned-rep-seqs_filt_masked_midroot-tree.qza')

# updated metadata; only samples with TimeTree host phylogeny
## SEE: http://127.0.0.1:9705/notebooks/notebooks/animal/phylogeny/TT/1_TimeTree_all-metadata.ipynb
metadata_file = '/ebio/abt3_projects/Georg_animal_feces/data/mapping/unified_metadata_complete_20180625_TT-filt.txt'

conda_env = 'qiime2'

# Init

In [2]:
library(dplyr)
library(tidyr)
library(phyloseq)
source('/ebio/abt3_projects/Georg_animal_feces/code/init.R') 


Attaching package: ‘dplyr’

The following objects are masked from ‘package:stats’:

    filter, lag

The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union



In [3]:
dir.create(work_dir, showWarnings = FALSE)
setwd(work_dir)

In [4]:
set.seed(48321)

# Converting qiime2 files

In [5]:
file.exists(SV_file) %>% print
file.exists(tax_file) %>% print
file.exists(tree_file) %>% print
file.exists(metadata_file) %>% print

[1] TRUE
[1] TRUE
[1] TRUE
[1] TRUE


## OTU table

In [6]:
# coverting artifact to biom
cmd = sprintf('qiime tools export --output-dir . %s', SV_file)
bash_job(cmd, conda_env)

character(0)

In [7]:
# converting biom to tsv
cmd = sprintf('biom convert -i feature-table.biom -o otu_table.tsv --to-tsv')
bash_job(cmd, conda_env)

character(0)

## Taxonomy

In [8]:
# coverting artifact to biom
cmd = sprintf('qiime tools export --output-dir . %s', tax_file)
bash_job(cmd, conda_env)

character(0)

In [9]:
# formatting taxonomy
exe = '/ebio/abt3_projects/Georg_animal_feces/code/taxonomy_format.py'
cmd = sprintf('%s %s > %s', exe, 'taxonomy.tsv', 'taxonomy_L0-6.tsv')
bash_job(cmd, conda_env)

character(0)

## Tree

In [10]:
# coverting artifact to biom
cmd = sprintf('qiime tools export --output-dir . %s', tree_file)
bash_job(cmd, conda_env)

character(0)

# Load

In [11]:
# otu table
physeq_otu = read.delim('otu_table.tsv', sep='\t', skip=1, comment.char='~')
rownames(physeq_otu) = physeq_otu$X.OTU.ID
physeq_otu$X.OTU.ID = NULL
colnames(physeq_otu) = gsub(' $', '', colnames(physeq_otu))

physeq_otu %>% status

[1] 39903   312


,X1.Eurasian.Beaver,X101.Horse,X102.Kulan,X104.European.Robin,X109.Red.Fox,X110.Noctule,X111.West.European.Hedgehog,X112.Eurasian.Sparrowhawk,X113.Eurasian.Woodcock,X116.Common.Kestrel,⋯,F46.Ref3.Red.Deer,F47.Red.Deer,F48.Red.Deer,F53.Mouflon,F66.Wild.Boar,F68.Red.Deer,F69.Red.Deer,F70.Red.Deer,F80.Red.Deer,F90.Domestic.Dog
294344edcaecca464652951ce825a77d,0,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
7a252295ae858fa6f9423b5166344a21,0,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
e49c6b87a2c49db4687078e754ca94da,0,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0


In [12]:
# metadata
metadata = read.delim(metadata_file, sep='\t') %>%
    mutate(X.SampleID = gsub('^([0-9])', 'X\\1', X.SampleID)) %>%
    filter(!grepl('\\.fail\\.', X.SampleID))
rownames(metadata) = metadata$X.SampleID

metadata %>% status

[1] 390 170


,X.SampleID,BarcodeSequence,LinkerPrimerSequence,Primer,ReversePrimer,Barcode_ID,sample_number,host_subject_id,host_german_name,host_common_name,⋯,wellplate_position_Illumina,Cornell_Dilution_DNA_Illumina,Num_Illumina,Num,sorting_454,comments_additional,DNA_box_20180622,sample_vol_20180622,sample_label_20180622,Description
X1.Eurasian.Beaver,X1.Eurasian.Beaver,ACTCACGGTATG,CATGCTGCCTCCCGTAGGAGT,GCCTCCCTCGCGCCATCAG,AGAGTTTGATCCTGGCTCAG,338R_BC0049,1,1,Biber,Eurasian.Beaver,⋯,B12,4,120,18,18,NA,1,20,1,NA
X2.Raccoon.Dog,X2.Raccoon.Dog,ACTCAGATACTC,CATGCTGCCTCCCGTAGGAGT,GCCTCCCTCGCGCCATCAG,AGAGTTTGATCCTGGCTCAG,338R_BC0050,2,2,Marderhund,Raccoon.Dog,⋯,C1,4,121,19,19,NA,1,20,2,NA
X3.Alpine.Chamois,X3.Alpine.Chamois,ACTCGCACAGGA,CATGCTGCCTCCCGTAGGAGT,GCCTCCCTCGCGCCATCAG,AGAGTTTGATCCTGGCTCAG,338R_BC0052,3,3,Gams,Alpine.Chamois,⋯,C2,4,122,20,20,NA,1,20,3,NA


In [13]:
# taxonomy 
physeq_tax = read.delim('taxonomy_L0-6.tsv', sep='\t') %>%
    filter(Feature.ID %in% rownames(physeq_otu))
rownames(physeq_tax) = physeq_tax$Feature.ID
physeq_tax$Feature.ID = NULL
physeq_tax = physeq_tax %>% 
    dplyr::select(-Confidence)

physeq_tax %>% status

[1] 39903     7


,Domain,Phylum,Class,Order,Family,Genus,Species
0001fc475a4ad532d85da4d26068f1ec,Bacteria,Bacteroidetes,Bacteroidia,Bacteroidales,Rikenellaceae,Rikenella,uncultured bacterium
000288a359abcc5cfb8982766cfd635a,Bacteria,Firmicutes,Clostridia,Clostridiales,Ruminococcaceae,uncultured,unclassified
000357daf6f2a70a630804dfa4e3cd32,Bacteria,Tenericutes,Mollicutes,RF9,uncultured bacterium,,


In [14]:
# tree 
tree = ape::read.tree('tree.nwk')
tree


Phylogenetic tree with 39903 tips and 39154 internal nodes.

Tip labels:
	0df8b90209df33b4c3f5a0a842f65718, ea2df9bcd7f6a10e0fa509d237d19c6d, f0adce016b4c9bba1532f6f1682d4d64, 130b094f31455943fb20899a36af8afd, 34f34bc3bffc1ae78011e33f6bba2dc0, b9b9562a0f34ad60c552f7f46d7a4b3b, ...
Node labels:
	root, 0.947, 0.636, 0.838, 0.962, 0.829, ...

Rooted; includes branch lengths.

## Checks

In [15]:
# rooted tree?
ape::is.rooted(tree)

[1] TRUE

# Formatting metadata

In [16]:
# multiple samples per individual?
metadata %>%
    group_by(host_subject_id) %>%
    summarize(n=n()) %>%
    ungroup() %>%
    filter(n > 1)

host_subject_id,n
142,5
195,6
204,8
313,12


In [17]:
# check that these samples are indeed from the same host
metadata %>%
    group_by(host_subject_id) %>%
    mutate(n=n()) %>%
    ungroup() %>%
    filter(n > 1) %>%
    dplyr::select(X.SampleID, lat_lon, country, collection_date)

X.SampleID,lat_lon,country,collection_date
X142.Cattle,50.59549 12.6386475,Austria,6/12/09
X144.Cattle,50.59549 12.6386475,Austria,6/17/09
X146.Cattle,50.59549 12.6386475,Austria,7/1/09
X148.Cattle,50.59549 12.6386475,Austria,7/21/09
X150.Cattle,50.59549 12.6386475,Austria,8/12/09
X195.Domestic.Cat,48.20833 16.373064,Austria,10/15/12
X196.Domestic.Cat,48.20833 16.373064,Austria,10/31/12
X204.Red.Sheep,47.726944 16.081667,Austria,1/10/12
X205.Red.Sheep,47.726944 16.081667,Austria,2/6/12
X206.Red.Sheep,47.726944 16.081667,Austria,2/29/12


In [18]:
# random selection of one sample per host
metadata = metadata %>%
    group_by(host_subject_id) %>%
    sample_n(1) %>%
    ungroup()

metadata %>% status

[1] 363 170


X.SampleID,BarcodeSequence,LinkerPrimerSequence,Primer,ReversePrimer,Barcode_ID,sample_number,host_subject_id,host_german_name,host_common_name,⋯,wellplate_position_Illumina,Cornell_Dilution_DNA_Illumina,Num_Illumina,Num,sorting_454,comments_additional,DNA_box_20180622,sample_vol_20180622,sample_label_20180622,Description
X1.Eurasian.Beaver,ACTCACGGTATG,CATGCTGCCTCCCGTAGGAGT,GCCTCCCTCGCGCCATCAG,AGAGTTTGATCCTGGCTCAG,338R_BC0049,1,1,Biber,Eurasian.Beaver,⋯,B12,4,120,18,18,NA,1,20,1,NA
X10.House.Mouse,ACTGTGACTTCA,CATGCTGCCTCCCGTAGGAGT,GCCTCCCTCGCGCCATCAG,AGAGTTTGATCCTGGCTCAG,338R_BC0058,10,10,Hausmaus,House.Mouse,⋯,C8,4,128,27,27,NA,1,20,10,NA
X101.Horse,ACGACGTCTTAG,CATGCTGCCTCCCGTAGGAGT,GCCTCCCTCGCGCCATCAG,AGAGTTTGATCCTGGCTCAG,338R_BC0030,101,101,Hauspferd,Horse,⋯,E8,16,344,160,160,NA,2,20,101,NA


In [19]:
# lat/long
metadata = metadata %>%
    separate(lat_lon, c('latitude', 'longitude'), sep=' ') %>%
    mutate(latitude = latitude %>% as.Num,
           longitude = longitude %>% as.Num) 

metadata[,c('latitude', 'longitude')] %>% summary %>% print

metadata %>% status

    latitude        longitude     
 Min.   :-35.83   Min.   :-84.10  
 1st Qu.: 47.06   1st Qu.: 15.47  
 Median : 48.18   Median : 16.37  
 Mean   : 36.89   Mean   : 25.05  
 3rd Qu.: 48.24   3rd Qu.: 16.80  
 Max.   : 55.29   Max.   :140.12  
[1] 363 171


X.SampleID,BarcodeSequence,LinkerPrimerSequence,Primer,ReversePrimer,Barcode_ID,sample_number,host_subject_id,host_german_name,host_common_name,⋯,wellplate_position_Illumina,Cornell_Dilution_DNA_Illumina,Num_Illumina,Num,sorting_454,comments_additional,DNA_box_20180622,sample_vol_20180622,sample_label_20180622,Description
X1.Eurasian.Beaver,ACTCACGGTATG,CATGCTGCCTCCCGTAGGAGT,GCCTCCCTCGCGCCATCAG,AGAGTTTGATCCTGGCTCAG,338R_BC0049,1,1,Biber,Eurasian.Beaver,⋯,B12,4,120,18,18,NA,1,20,1,NA
X10.House.Mouse,ACTGTGACTTCA,CATGCTGCCTCCCGTAGGAGT,GCCTCCCTCGCGCCATCAG,AGAGTTTGATCCTGGCTCAG,338R_BC0058,10,10,Hausmaus,House.Mouse,⋯,C8,4,128,27,27,NA,1,20,10,NA
X101.Horse,ACGACGTCTTAG,CATGCTGCCTCCCGTAGGAGT,GCCTCCCTCGCGCCATCAG,AGAGTTTGATCCTGGCTCAG,338R_BC0030,101,101,Hauspferd,Horse,⋯,E8,16,344,160,160,NA,2,20,101,NA


In [20]:
# time period
metadata = metadata %>%
    mutate(collection_date = collection_date %>% as.character,
           collection_date = ifelse(collection_date == 'Autumn 2013', '10/1/13', collection_date)) 

metadata$collection_date %>% summary

   Length     Class      Mode 
      363 character character 

In [21]:
# striping whitespace from labels
tws = function(x){
    if(class(x) == 'numeric'){
        return(x)
    } else {
        x = sapply(as.character(x), function(y) trimws(y, which = 'both'))
        return(x)
    }
}

                   
metadata = metadata %>% apply(2, tws) %>%
    as.data.frame              
rownames(metadata) = metadata$X.SampleID

# status
metadata %>% status

[1] 363 171


,X.SampleID,BarcodeSequence,LinkerPrimerSequence,Primer,ReversePrimer,Barcode_ID,sample_number,host_subject_id,host_german_name,host_common_name,⋯,wellplate_position_Illumina,Cornell_Dilution_DNA_Illumina,Num_Illumina,Num,sorting_454,comments_additional,DNA_box_20180622,sample_vol_20180622,sample_label_20180622,Description
X1.Eurasian.Beaver,X1.Eurasian.Beaver,ACTCACGGTATG,CATGCTGCCTCCCGTAGGAGT,GCCTCCCTCGCGCCATCAG,AGAGTTTGATCCTGGCTCAG,338R_BC0049,1,1,Biber,Eurasian.Beaver,⋯,B12,4,120,18,18,NA,1,20,1,NA
X10.House.Mouse,X10.House.Mouse,ACTGTGACTTCA,CATGCTGCCTCCCGTAGGAGT,GCCTCCCTCGCGCCATCAG,AGAGTTTGATCCTGGCTCAG,338R_BC0058,10,10,Hausmaus,House.Mouse,⋯,C8,4,128,27,27,NA,1,20,10,NA
X101.Horse,X101.Horse,ACGACGTCTTAG,CATGCTGCCTCCCGTAGGAGT,GCCTCCCTCGCGCCATCAG,AGAGTTTGATCCTGGCTCAG,338R_BC0030,101,101,Hauspferd,Horse,⋯,E8,16,344,160,160,NA,2,20,101,NA


In [22]:
# summary
metadata %>%
    dplyr::select(class, order, family, genus,
                  diet, diet_vertebrate, diet_invertebrate, diet_fruit,
                  diet_flowers_nectar_pollen, diet_leaves_branches_bark_buds, diet_seeds,
                  diet_grass_waterplants, diet_roots_tubers,
                  digestive_system, digestive_system_detail, 
                  ruminant, pseudoruminants,
                  habitat_detail, habitat_detail_1, habitat_detail_2,
                  latitude, longitude,
                  sampling_method, captive_wild, wild_domesticated) %>%
    summary

            class               order            family        genus    
 Actinopterygii: 35   Artiodactyla : 54   Bovidae   : 21   Cervus : 14  
 Amphibia      : 10   Carnivora    : 43   Cyprinidae: 20   Equus  : 11  
 Aves          : 86   Passeriformes: 29   Cervidae  : 18   Lepus  : 11  
 Mammalia      :218   Rodentia     : 29   Leporidae : 15   Canis  :  7  
 Reptilia      : 14   Cypriniformes: 20   Felidae   : 13   Felis  :  7  
                      Primates     : 20   Canidae   : 11   Homo   :  7  
                      (Other)      :168   (Other)   :265   (Other):306  
        diet             diet_vertebrate             diet_invertebrate
 carnivore:116   no              :233    no                   :136    
 herbivore:112   yes             :129    yes                  :225    
 omnivore :135   yes on occassion:  1    yes mainly young ones:  1    
                                         yes on occassion     :  1    
                                                             

# Checking overlap

In [23]:
# in metadata, but not in OTU table?
setdiff(colnames(physeq_otu), rownames(metadata)) %>% print
cat('----\n')
setdiff(rownames(metadata), colnames(physeq_otu)) %>% print

 [1] "X142.Cattle"               "X146.Cattle"              
 [3] "X148.Cattle"               "X150.Cattle"              
 [5] "X206.Red.Sheep"            "X208.Red.Sheep"           
 [7] "X213.Goose"                "X214.Goose"               
 [9] "X216.Goose"                "X316.Red.Deer"            
[11] "X386.Scalloped.Hammerhead" "X388.Bull.Ray"            
[13] "X397.Penguin"              "F46.Ref1.Red.Deer"        
[15] "F46.Ref2.Red.Deer"         "F46.Ref4.Red.Deer"        
[17] "X195.Domestic.Cat"         "X196.Domestic.Cat"        
[19] "X205.Red.Sheep"            "X209.Red.Sheep"           
[21] "X211.Red.Sheep"            "X212.Goose"               
[23] "X215.Goose"                "X218.Goose"               
[25] "X219.Goose"                "X309.Domestic.Cat"        
[27] "X311.Domestic.Cat"         "X312.Domestic.Cat"        
[29] "X314.Red.Deer"             "X317.Red.Deer"            
[31] "X318.Red.Deer"             "X319.Red.Deer"            
[33] "X320.Red.Deer"    

## Filtering to intersection

In [24]:
# intersect of samples
x = intersect(colnames(physeq_otu), rownames(metadata))
x %>% length %>% print

[1] 273


In [25]:
# filtering samples from otu table
physeq_otu = physeq_otu[,x]
physeq_otu %>% status

[1] 39903   273


,X1.Eurasian.Beaver,X101.Horse,X102.Kulan,X104.European.Robin,X109.Red.Fox,X110.Noctule,X111.West.European.Hedgehog,X112.Eurasian.Sparrowhawk,X113.Eurasian.Woodcock,X116.Common.Kestrel,⋯,F45.Red.Deer,F47.Red.Deer,F48.Red.Deer,F53.Mouflon,F66.Wild.Boar,F68.Red.Deer,F69.Red.Deer,F70.Red.Deer,F80.Red.Deer,F90.Domestic.Dog
294344edcaecca464652951ce825a77d,0,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
7a252295ae858fa6f9423b5166344a21,0,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
e49c6b87a2c49db4687078e754ca94da,0,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0


In [26]:
# filtering metdata
metadata = metadata[x,]
metadata %>% status

[1] 273 171


,X.SampleID,BarcodeSequence,LinkerPrimerSequence,Primer,ReversePrimer,Barcode_ID,sample_number,host_subject_id,host_german_name,host_common_name,⋯,wellplate_position_Illumina,Cornell_Dilution_DNA_Illumina,Num_Illumina,Num,sorting_454,comments_additional,DNA_box_20180622,sample_vol_20180622,sample_label_20180622,Description
X1.Eurasian.Beaver,X1.Eurasian.Beaver,ACTCACGGTATG,CATGCTGCCTCCCGTAGGAGT,GCCTCCCTCGCGCCATCAG,AGAGTTTGATCCTGGCTCAG,338R_BC0049,1,1,Biber,Eurasian.Beaver,⋯,B12,4,120,18,18,NA,1,20,1,NA
X101.Horse,X101.Horse,ACGACGTCTTAG,CATGCTGCCTCCCGTAGGAGT,GCCTCCCTCGCGCCATCAG,AGAGTTTGATCCTGGCTCAG,338R_BC0030,101,101,Hauspferd,Horse,⋯,E8,16,344,160,160,NA,2,20,101,NA
X102.Kulan,X102.Kulan,ACGAGTGCTATC,CATGCTGCCTCCCGTAGGAGT,GCCTCCCTCGCGCCATCAG,AGAGTTTGATCCTGGCTCAG,338R_BC0031,102,102,Khulan,Kulan,⋯,E9,16,345,161,161,NA,2,20,102,NA


In [27]:
# otu overlap with tree
setdiff(rownames(physeq_otu), rownames(physeq_tax)) %>% print
setdiff(rownames(physeq_otu), tree$tip.label) %>% print

character(0)
character(0)


# Making phyloseq obj

In [28]:
physeq = phyloseq(
                  otu_table(physeq_otu %>% as.matrix, taxa_are_rows=TRUE),
                  sample_data(metadata),
                  tax_table(physeq_tax %>% as.matrix),
                  phy_tree(tree)
                 )
physeq

phyloseq-class experiment-level object
otu_table()   OTU Table:         [ 39903 taxa and 273 samples ]
sample_data() Sample Data:       [ 273 samples by 171 sample variables ]
tax_table()   Taxonomy Table:    [ 39903 taxa by 7 taxonomic ranks ]
phy_tree()    Phylogenetic Tree: [ 39903 tips and 39154 internal nodes ]

## Writing object

In [29]:
physeq_file = file.path(work_dir, 'physeq.RDS')
saveRDS(physeq, file=physeq_file)
cat('File written:', physeq_file, '\n')

File written: /ebio/abt3_projects/Georg_animal_feces/data/16S/LLA/phyloseq//physeq.RDS 


# sessionInfo

In [30]:
sessionInfo()

R version 3.4.1 (2017-06-30)
Platform: x86_64-pc-linux-gnu (64-bit)
Running under: Ubuntu 16.04.4 LTS

Matrix products: default
BLAS: /ebio/abt3_projects/software/miniconda3/envs/py3_physeq/lib/R/lib/libRblas.so
LAPACK: /ebio/abt3_projects/software/miniconda3/envs/py3_physeq/lib/R/lib/libRlapack.so

locale:
 [1] LC_CTYPE=en_US.UTF-8       LC_NUMERIC=C              
 [3] LC_TIME=en_US.UTF-8        LC_COLLATE=en_US.UTF-8    
 [5] LC_MONETARY=en_US.UTF-8    LC_MESSAGES=en_US.UTF-8   
 [7] LC_PAPER=en_US.UTF-8       LC_NAME=C                 
 [9] LC_ADDRESS=C               LC_TELEPHONE=C            
[11] LC_MEASUREMENT=en_US.UTF-8 LC_IDENTIFICATION=C       

attached base packages:
[1] stats     graphics  grDevices utils     datasets  methods   base     

other attached packages:
[1] bindrcpp_0.2    phyloseq_1.22.3 tidyr_0.8.1     dplyr_0.7.4    

loaded via a namespace (and not attached):
 [1] pbdZMQ_0.3-1        tidyselect_0.2.4    repr_0.12.0        
 [4] reshape2_1.4.3      purrr_0.2.

In [31]:
bash_job('conda list', conda_env) %>% paste(collapse='\n') %>% cat

# packages in environment at /ebio/abt3_projects/software/miniconda3/envs/qiime2:
#
aioeasywebdav             2.2.0                    py35_0    conda-forge
aiohttp                   2.2.5            py35h8d7d2b0_0  
appdirs                   1.4.3                    py35_0  
asn1crypto                0.22.0                   py35_0    conda-forge
async-timeout             2.0.0            py35h12a94dc_0  
backports                 1.0                      py35_1    conda-forge
backports.functools_lru_cache 1.4                      py35_1    conda-forge
bcrypt                    3.1.4            py35ha35c455_0  
bioconductor-biobase      2.34.0                 r3.3.2_0    bioconda
bioconductor-biocgenerics 0.22.0                 r3.3.2_0    bioconda
bioconductor-biocparallel 1.6.6                  r3.3.2_1    bioconda
bioconductor-biostrings   2.42.1                 r3.3.2_0    bioconda
bioconductor-dada2        1.4.0                  r3.3.2_0    bioconda
bioconductor-genomeinfodb 1.10